In [1]:
pwd

'/content'

In [2]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Load the dataset
df=pd.read_csv("./sample_data/drugsCom.csv",encoding="ISO-8859-1")

In [4]:
df.tail()

,MedicineName\t,Condition,Review,rating,date,usefulCount
128745,Zyvox,Bacterial Infection,"""It didn&#039;t work as fast as I thought it w...",6.0,"April 17, 2016",1.0
128746,Zyvox,Skin and Structure Infection,"""My father is taking this medicine for Celluli...",5.0,"July 22, 2009",8.0
128747,ZzzQuil,Insomnia,"""Gave me rapid heart beats !!!!!!!!!""",1.0,"May 26, 2017",1.0
128748,ZzzQuil,Insomnia,"""I purchased the zzzquil and it did not work n...",1.0,"January 13, 2016",17.0
128749,ZzzQuil,Insomnia,"""I usually work a 10 hr shift in the afternoon...",4.0,"May 27, 2017",3.0


In [5]:
df.shape

(128750, 6)

In [ ]:
#Data Cleaning (Normalizing the text)
#Make text lowercase, remove punctuation and numbers present the reviews.

In [6]:
import re #regular expression
import string
def clean_Review(Review):
    Review=Review.lower()
    Review = re.sub('[0-9]', '', Review)
    Review = Review.translate(str.maketrans('', '', string.punctuation))
    return Review

clean = lambda x: clean_Review(x)

In [7]:
df['Review'] = df.Review.apply(clean)

In [8]:
df.tail()

,MedicineName\t,Condition,Review,rating,date,usefulCount
128745,Zyvox,Bacterial Infection,it didnt work as fast as i thought it would ve...,6.0,"April 17, 2016",1.0
128746,Zyvox,Skin and Structure Infection,my father is taking this medicine for cellulit...,5.0,"July 22, 2009",8.0
128747,ZzzQuil,Insomnia,gave me rapid heart beats,1.0,"May 26, 2017",1.0
128748,ZzzQuil,Insomnia,i purchased the zzzquil and it did not work no...,1.0,"January 13, 2016",17.0
128749,ZzzQuil,Insomnia,i usually work a hr shift in the afternoon bu...,4.0,"May 27, 2017",3.0


In [9]:
#Checking Null values
df.isnull().sum()

MedicineName\t        0
Condition           609
Review                0
rating            14689
date              14689
usefulCount       14689
dtype: int64

In [10]:
#Droping the Null Rows
df=df.dropna()

In [11]:
df.shape

(113501, 6)

In [12]:
#Checking Null values
df.isnull().sum()
#NO Null values

MedicineName\t    0
Condition         0
Review            0
rating            0
date              0
usefulCount       0
dtype: int64

In [13]:
#No of Medicines in dataset
df["MedicineName\t"].nunique()

2335

In [14]:
#Find the duplicates
df.iloc[:,0:4].duplicated().sum()

43

In [16]:
#Droping duplicates
df=df.drop_duplicates(subset=["MedicineName\t","Condition","Review","rating"],keep='first')

In [17]:
df.iloc[:,0:4].duplicated().sum()
#No duplicate values

0

In [18]:
df.shape

(113458, 6)

In [20]:
#expanding contractions 
!pip install contractions

In [ ]:
import contractions
#cont = contractions('GoogleNews-vectors-negative300.bin')
df['Updated_Review'] = df['Review'].apply(lambda x:[contractions.fix(word) for word in x.split()])

In [25]:
df.tail(2)

,MedicineName\t,Condition,Review,rating,date,usefulCount,Updated_Review
128748,ZzzQuil,Insomnia,i purchased the zzzquil and it did not work no...,1.0,"January 13, 2016",17.0,"[i, purchased, the, zzzquil, and, it, did, not..."
128749,ZzzQuil,Insomnia,i usually work a hr shift in the afternoon bu...,4.0,"May 27, 2017",3.0,"[i, usually, work, a, hr, shift, in, the, afte..."


In [ ]:
#Coverting Updated_Review column into string format for further preprocessing
df['Updated_Review'] = [' '.join(map(str, l)) for l in df['Updated_Review']]

In [28]:
df.tail()

,MedicineName\t,Condition,Review,rating,date,usefulCount,Updated_Review
128745,Zyvox,Bacterial Infection,it didnt work as fast as i thought it would ve...,6.0,"April 17, 2016",1.0,it did not work as fast as i thought it would ...
128746,Zyvox,Skin and Structure Infection,my father is taking this medicine for cellulit...,5.0,"July 22, 2009",8.0,my father is taking this medicine for cellulit...
128747,ZzzQuil,Insomnia,gave me rapid heart beats,1.0,"May 26, 2017",1.0,gave me rapid heart beats
128748,ZzzQuil,Insomnia,i purchased the zzzquil and it did not work no...,1.0,"January 13, 2016",17.0,i purchased the zzzquil and it did not work no...
128749,ZzzQuil,Insomnia,i usually work a hr shift in the afternoon bu...,4.0,"May 27, 2017",3.0,i usually work a hr shift in the afternoon but...


In [29]:
# English language detection using standard facebook pretrained model
pretrained_model = "lid.176.bin" 

In [ ]:
#fastText is a library for efficient learning of word representations and sentence classification.
!pip install fasttext

In [ ]:
import fasttext
model = fasttext.load_model('./sample_data/lid.176.bin')

langs = []
for sent in df['Updated_Review']:
  lang = model.predict(sent)[0]
  langs.append(str(lang)[11:13])

df['language'] = langs

In [33]:
pd.value_counts(df['language'])

en    113408
no        10
sv         8
fr         7
it         5
ce         4
nl         4
de         3
es         2
vi         2
ca         2
da         1
pl         1
pt         1
Name: language, dtype: int64

In [34]:
df.tail()

,MedicineName\t,Condition,Review,rating,date,usefulCount,Updated_Review,language
128745,Zyvox,Bacterial Infection,it didnt work as fast as i thought it would ve...,6.0,"April 17, 2016",1.0,it did not work as fast as i thought it would ...,en
128746,Zyvox,Skin and Structure Infection,my father is taking this medicine for cellulit...,5.0,"July 22, 2009",8.0,my father is taking this medicine for cellulit...,en
128747,ZzzQuil,Insomnia,gave me rapid heart beats,1.0,"May 26, 2017",1.0,gave me rapid heart beats,en
128748,ZzzQuil,Insomnia,i purchased the zzzquil and it did not work no...,1.0,"January 13, 2016",17.0,i purchased the zzzquil and it did not work no...,en
128749,ZzzQuil,Insomnia,i usually work a hr shift in the afternoon bu...,4.0,"May 27, 2017",3.0,i usually work a hr shift in the afternoon but...,en


In [35]:
#removing reviews from other languages
df = df.loc[df['language'] == 'en']

In [36]:
df.tail(2)

,MedicineName\t,Condition,Review,rating,date,usefulCount,Updated_Review,language
128748,ZzzQuil,Insomnia,i purchased the zzzquil and it did not work no...,1.0,"January 13, 2016",17.0,i purchased the zzzquil and it did not work no...,en
128749,ZzzQuil,Insomnia,i usually work a hr shift in the afternoon bu...,4.0,"May 27, 2017",3.0,i usually work a hr shift in the afternoon but...,en


In [37]:
df = df.drop(['language'], axis = 1)
df.tail(2)

,MedicineName\t,Condition,Review,rating,date,usefulCount,Updated_Review
128748,ZzzQuil,Insomnia,i purchased the zzzquil and it did not work no...,1.0,"January 13, 2016",17.0,i purchased the zzzquil and it did not work no...
128749,ZzzQuil,Insomnia,i usually work a hr shift in the afternoon bu...,4.0,"May 27, 2017",3.0,i usually work a hr shift in the afternoon but...


In [ ]:
#Tokenize the review
import nltk
nltk.download('punkt')

In [40]:
from nltk import word_tokenize
df['Updated_Review']=df['Updated_Review'].apply(word_tokenize)

In [41]:
df.tail(2)

,MedicineName\t,Condition,Review,rating,date,usefulCount,Updated_Review
128748,ZzzQuil,Insomnia,i purchased the zzzquil and it did not work no...,1.0,"January 13, 2016",17.0,"[i, purchased, the, zzzquil, and, it, did, not..."
128749,ZzzQuil,Insomnia,i usually work a hr shift in the afternoon bu...,4.0,"May 27, 2017",3.0,"[i, usually, work, a, hr, shift, in, the, afte..."


In [ ]:
#Removing StopWords

In [ ]:
nltk.download('stopwords')

In [44]:
#Remove Stopwords
from nltk.corpus import stopwords
df['Updated_Review'] = df['Updated_Review'].apply(lambda x:[w for w in x if w not in stopwords.words('english')])

In [45]:
df.tail(2)

,MedicineName\t,Condition,Review,rating,date,usefulCount,Updated_Review
128748,ZzzQuil,Insomnia,i purchased the zzzquil and it did not work no...,1.0,"January 13, 2016",17.0,"[purchased, zzzquil, work, bit, would, recomme..."
128749,ZzzQuil,Insomnia,i usually work a hr shift in the afternoon bu...,4.0,"May 27, 2017",3.0,"[usually, work, hr, shift, afternoon, boss, ch..."


In [ ]:
 nltk.download('wordnet')

In [47]:
from nltk import WordNetLemmatizer
wordnet = WordNetLemmatizer()
df['Updated_Review'] = df['Updated_Review'].apply(lambda x: [wordnet.lemmatize(w) for w in x]) 

In [48]:
df.tail(2)

,MedicineName\t,Condition,Review,rating,date,usefulCount,Updated_Review
128748,ZzzQuil,Insomnia,i purchased the zzzquil and it did not work no...,1.0,"January 13, 2016",17.0,"[purchased, zzzquil, work, bit, would, recomme..."
128749,ZzzQuil,Insomnia,i usually work a hr shift in the afternoon bu...,4.0,"May 27, 2017",3.0,"[usually, work, hr, shift, afternoon, bos, cha..."


In [52]:
df.rename(columns={'MedicineName\t': 'MedicineName'}, inplace=True)

In [53]:
df.tail()

,MedicineName,Condition,Review,rating,date,usefulCount,Updated_Review
128745,Zyvox,Bacterial Infection,it didnt work as fast as i thought it would ve...,6.0,"April 17, 2016",1.0,"[work, fast, thought, would, expensive, made, ..."
128746,Zyvox,Skin and Structure Infection,my father is taking this medicine for cellulit...,5.0,"July 22, 2009",8.0,"[father, taking, medicine, cellulitis, started..."
128747,ZzzQuil,Insomnia,gave me rapid heart beats,1.0,"May 26, 2017",1.0,"[gave, rapid, heart, beat]"
128748,ZzzQuil,Insomnia,i purchased the zzzquil and it did not work no...,1.0,"January 13, 2016",17.0,"[purchased, zzzquil, work, bit, would, recomme..."
128749,ZzzQuil,Insomnia,i usually work a hr shift in the afternoon bu...,4.0,"May 27, 2017",3.0,"[usually, work, hr, shift, afternoon, bos, cha..."


In [54]:
#Converting df to csv
df.to_csv('Preprocessed_Drugs.csv')

In [ ]:
#Note:Merged Vinayak code with Mine